Se compara el desempeño entre los siguientes modelos/solvers:

* Modelo base
* Modelo bloque Transformer
* Modelo bloque Transformer * 6
* Modelo con encoding posicional
* Greedy
* OR-Tools


## Descarga de librerías

In [40]:
# Descarga de librerías

# Linux only, doesn't work on windows
# ! python -c "import ortools" 2>/dev/null  && echo "OR-Tools is already installed" || pip install ortools -q
# ! [[ ! -d eda ]]  && echo "Downloading eda repo" && curl -L  https://github.com/rilianx/eda/archive/refs/heads/main.tar.gz | tar xzvf - && mv eda-main eda
# import torch
# !pip install tqdm

In [41]:
# ! curl -LO https://github.com/rilianx/eda/archive/refs/heads/main.tar.gz
# ! 7z x main.tar.gz
# ! 7z x main.tar
# ! move eda-main eda
# ! del main.tar.gz
# ! del main.tar

## state2vecSeq


In [59]:
# Generación de datos


from copy import deepcopy
import random
import math

import numpy as np
from torch.nn.functional import one_hot
from eda.TSP import TSP_Instance, TSP_Environment, TSP_State
from eda.solveTSP_v2 import solve
env = TSP_Environment

def distance(punto1, punto2):
    return math.sqrt((punto1[0] - punto2[0])**2 + (punto1[1] - punto2[1])**2)

# función para transformar un estado tsp en una secuencia de vectores
# para el modelo basado en capas de atención
def state2vecSeq(self):
    # creamos dos diccionarios para mantenre un mapeo de los
    # movimientos con los índices de la secuencia del modelo de aprendizaje

    city_locations = self.inst_info.city_locations

    idx2move = dict()
    move2idx = dict()
    origin = city_locations[self.visited[-1]]
    destination = city_locations[self.visited[0]]

    origin_dist = 0.0
    dest_dist = distance(origin, destination)

    seq = [list(origin) + [1,0] + [origin_dist, dest_dist], # Última ciudad visitada (origen)
           list(destination) + [0, 1] + [dest_dist, 0.0]]  # Ciudad final

    idx2move[0] = None
    idx2move[1] = ("constructive", self.visited[0])
    move2idx[self.visited[0]] = 1

    idx = 2
    for i in self.not_visited:
        point = list(city_locations[i])
        origin_dist = distance( point, origin)
        dest_dist = distance( point, destination)
        city_vector = point + [0, 0] + [origin_dist, dest_dist] # Otras ciudades

        seq.append(city_vector)
        idx2move[idx] = ("constructive", i)
        move2idx[i] = idx
        idx += 1

    return seq, idx2move, move2idx




## Meta clase para definir modelos y ciclo de entrenamiento

Se definen hiperparámetros de los modelos a evaluar

In [60]:
## Todos los modelos serán entrenados con el mismo dataset
# X: [20000, 11, 6], Y: [20000, 11]
# donde X: (nb_sample, max_cities + 1, param_count), Y: (nb_sample, max_cities+1)
import torch
from tqdm import tqdm
class Model:
    # El modelo se genera en el constructor y se guarda en self.model
    def __init__(self, 
         input_dim = 6,
         num_heads = 10,
         head_dim = 64,
         city_count = 50,
                 
         batch_size = 512,
         train_split = 0.5,
         nb_samples = 20000,
         epochs = 10):

        self.city_count = city_count # Número de ciudades a evaluar
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        ## Parámetros modelo
        self.input_dim = input_dim  # Dimensión de la entrada
        self.num_heads = num_heads  # Número de cabezas en la atención multi-cabeza
        self.head_dim = head_dim  # Dimensión de cada cabeza
        ## Parámetros entrenamiento
        self.batch_size = batch_size
        self.train_split = train_split
        self.nb_samples = nb_samples
        self.epochs = 10

        
        self.model = None
    
    
    def load_model(self):
        raise NotImplementedError("La función 'load_model' debe ser declarada")
    def unload_model(self):
        raise NotImplementedError("La función 'unload_model' debe ser implementada");

    def train(self, x, y):
        raise NotImplementedError("La función 'train' debe ser declarada")
      

    def generate_data(self, use_progress_bar=False):
        X = []  # Lista para almacenar las secuencias de entrada
        Y = []  # Lista para almacenar las etiquetas objetivo (las siguientes ciudades a visitar)
        seq_len = self.city_count + 1  # Longitud de la secuencia, ajustada para incluir una ciudad extra
        
        # If the flag is set, initialize the progress bar
        pbar = tqdm(total=self.nb_samples, desc="Generating data", unit="sample", position=0, leave=True) if use_progress_bar else None
        
        # Bucle para generar datos hasta alcanzar el número deseado de muestras
        while True:
            # 1. Generamos instancia aleatoria
            n_cities = self.city_count
            dim = 2  # Dimensión para las coordenadas de la ciudad (2D: x, y)
            city_points = np.random.rand(n_cities, dim)  # Generar puntos aleatorios para las ciudades
            inst_info = TSP_Instance(city_points)
    
            # 2. Resolvemos TSP usando algoritmo tradicional
            solution = solve(city_points)  # Resolver el TSP y obtener un estado final
    
            # 3. Iteramos sobre los movimientos de la solución final para generar varias muestras:
            # estado (X) -> movimiento (Y)
            current_state = TSP_State(inst_info)
            env.state_transition(current_state, ("constructive", solution.visited[0]))
            samples_per_sol = self.city_count - 1  # Número máximo de muestras por solución
            
            for move in [("constructive", city) for city in solution.visited[1:]]:
                seq, _, move2idx = state2vecSeq(current_state)  # Convertir el estado actual a secuencia vectorizada
    
                X.append(torch.tensor(seq))  # Añadir la secuencia a X
                Y.append(one_hot(torch.tensor(move2idx[move[1]]), num_classes=seq_len))
                #Y.append(to_categorical(move2idx[move[1]], num_classes=seq_len))  # Añadir el movimiento como categoría a Y
    
                env.state_transition(current_state, move)  # Hacer la transición al siguiente estado
    
                # Actualizar el progreso de la barra si se está usando
                if use_progress_bar:
                    pbar.update(1)
    
                # Condiciones de parada basadas en el número de ciudades visitadas/no visitadas o muestras generadas
                if len(current_state.visited) > samples_per_sol or len(X) >= self.nb_samples:
                    break
    
            # Romper el bucle externo si se ha alcanzado el número deseado de muestras
            if len(X) >= self.nb_samples:
                break
        
        # Close the progress bar if it was used
        if use_progress_bar:
            pbar.close()
    
        X_padded = torch.nn.utils.rnn.pad_sequence(X, batch_first=True)
        
        return X_padded, torch.stack(Y)


Model()

In [61]:
## Definición base de modelo para usar el mismo entrenamiento en los distintos modelos
import pandas as pd
import gc
from eda.agents import GreedyAgent, SingleAgentSolver
from tsp.model_eval_actions import ModelEvalActions
class TrainableModel(Model):
    def __init__(self):
        super().__init__()
        # Parámetros del modelo
    def unload_model(self):
        del model.model
        torch.cuda.empty_cache()
        gc.collect()

    def train(self, xt, yt, xv, yv, num_iter=-1, use_progress_bar=False):
        # self.load_model()
        # Asumiendo que X_padded y Y_stacked ya están definidos y son tensores de PyTorch
        trd = TensorDataset(xt, yt)
        ted = TensorDataset(xv, yv)
    
        # # Dividir el dataset en entrenamiento y prueba
        # train_size = int(self.train_split * len(dataset))
        # test_size = len(dataset) - train_size
        train_dataset, test_dataset = trd, ted
    
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)
    
        # Definir el modelo, la función de pérdida y el optimizador
        loss_function = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.model.parameters())
    
        # Initialize the DataFrame to store training results
        df = pd.DataFrame(columns=["Model Name", "cities", "iter", "Epoch",
                                   "Training Loss", "Training Accuracy",
                                   "Validation Loss", "Validation Accuracy"])
        dfb = pd.DataFrame(columns=["Model Name", "cities", "iter", "Epoch", "cost"])
    
        # Initialize the progress bar for epochs if required
        epoch_range = range(self.epochs)
        if use_progress_bar:
            epoch_range = tqdm(epoch_range, desc="Training Epochs", unit="epoch", position = 0, leave = True)
        
        print("Entrenando modelo...")
        for epoch in epoch_range:
            self.model.train()
            train_loss = 0
            correct = 0
            total = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()  # Limpia los gradientes
                outputs = self.model(X_batch)  # Obtenemos logits
                loss = loss_function(outputs, y_batch.argmax(dim=1))  # Calcular la pérdida
                loss.backward()  # Backward pass
                optimizer.step()  # Actualizar parámetros
                train_loss += loss.item() * X_batch.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch.argmax(dim=1)).sum().item()
    
            train_loss /= len(train_loader.dataset)
            train_accuracy = 100 * correct / total
    
            # Validación
            self.model.eval()
            validation_loss = 0
            correct = 0
            total = 0
            with torch.no_grad():
                for X_batch, y_batch in test_loader:
                    outputs = self.model(X_batch)
                    loss = loss_function(outputs, y_batch.argmax(dim=1))
                    validation_loss += loss.item() * X_batch.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    total += y_batch.size(0)
                    correct += (predicted == y_batch.argmax(dim=1)).sum().item()
    
            validation_loss /= len(test_loader.dataset)
            validation_accuracy = 100 * correct / total
    
            # Log results to DataFrame
            df = pd.concat([df, pd.DataFrame([{
                "Model Name": type(self).__name__,
                "cities": self.city_count,
                "iter": num_iter,
                "Epoch": epoch + 1,
                "Training Loss": train_loss,
                "Training Accuracy": train_accuracy,
                "Validation Loss": validation_loss,
                "Validation Accuracy": validation_accuracy
            }])], ignore_index=True)
    
            # print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
            # print(f'Epoch {epoch+1}, Val Loss: {validation_loss:.4f}, Val Accuracy: {validation_accuracy:.2f}%')
            instances = [
                TSP_Instance(np.random.rand(model.city_count, 2)) for _ in tqdm(
                    range(nb_eval), desc="Instances", unit="instance", position=0, leave=True
                )
            ]
            greedy = SingleAgentSolver(env, GreedyAgent(ModelEvalActions(model.model)))
            solutions = []

            for instance in tqdm(instances, desc="Solving Instances", unit="instance", position=0, leave=True):
                solution, *_ = greedy.solve(TSP_State(instance, visited=[0]))
                dfb = pd.concat([dfb, pd.DataFrame([{
                    "ModelName" : type(model).__name__,
                    "cities": model.city_count,
                    "Epoch": epoch,
                    "iter": fold,
                    "cost": solution.cost,                
            }]) ])
    
        # If tqdm was used, close the progress bar
        if use_progress_bar:
            epoch_range.close()
    
        # self.unload_model()
        return df, dfb;


In [62]:
## Modelo base

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, random_split

class BaseModel(TrainableModel):

    
    def load_model(self):
        self.model = BaseModel.CustomModel(self.input_dim, self.num_heads, self.head_dim)
        
    class CustomModel(nn.Module):
        def __init__(self, input_dim, num_heads, head_dim, dropout_rate=0.2):
            super(BaseModel.CustomModel, self).__init__()
            #self.seq_length = seq_length  # Asumiendo una longitud fija de secuencia para simplificar
            self.num_heads = num_heads
            self.head_dim = head_dim

            # Proyección de entrada
            self.input_projection = nn.Linear(input_dim, num_heads * head_dim)

            # Capa de atención multi-cabeza
            self.multihead_attention = nn.MultiheadAttention(embed_dim=num_heads * head_dim,
                                                            num_heads=num_heads,
                                                            dropout=dropout_rate)

            # Capas lineales individuales para cada posición de la secuencia
            # Esto es un cambio respecto al código original para aplicar una capa lineal por posición de salida
            self.positionwise_linear = nn.Linear(num_heads * head_dim, 1)

            # Capa de salida final, después de un flatten, para aplicar Softmax
            # Nota: Softmax se aplica después del flatten, por lo tanto no se define aquí como una capa pero sí en el forward

        def generate_attention_mask(self, x, padding_value=0):
            # Identificar posiciones de padding en x
            mask = (x.sum(dim=-1) == padding_value)  # Asumiendo que el padding se puede identificar sumando los valores de la característica y comparando con 0
            mask = mask.to(dtype=torch.bool)  # Convierte a bool para usar como máscara
            # PyTorch espera una máscara con True y False donde True indica donde aplicar la máscara
            return mask


        def forward(self, x, seq_lengths=10, return_probabilities=False):
            # x: [batch_size, seq_length, input_dim]
            x = x.float()

            max_len = x.shape[1]

            # Generar máscara de atención basada en las longitudes de las secuencias
            attn_mask = None

            # Aplicar proyección de entrada
            x_proj = self.input_projection(x)
            x_proj = x_proj.permute(1, 0, 2)  # Reordenar para multihead_attention: [seq_length, batch_size, num_heads*head_dim]


            # Aplicar atención multi-cabeza
            attn_output, _ = self.multihead_attention(x_proj, x_proj, x_proj, attn_mask=attn_mask)
            attn_output = attn_output.permute(1, 0, 2)  # Reordenar de vuelta: [batch_size, seq_length, num_heads*head_dim]

            # Aplicar la capa lineal posición por posición
            # Usamos una capa lineal que se aplica a cada vector de salida de la atención de forma independiente
            positionwise_output = self.positionwise_linear(attn_output)

            # Flatten
            flat_output = positionwise_output.view(positionwise_output.size(0), -1)  # [batch_size, seq_length]

            # Softmax
            if return_probabilities:
                output = F.softmax(flat_output, dim=-1)
                return output
            else: #return logits
                return flat_output


    
    def __init__(self):
        super().__init__()
    def load_model(self):
        self.model = BaseModel.CustomModel(input_dim=self.input_dim, num_heads=self.num_heads, head_dim=self.head_dim)
        self.model.to(self.device)



In [63]:
class TransformerBlockModel(TrainableModel):

  class CustomModel(nn.Module):
    class TransformerBlock(nn.Module):
        def __init__(self, input_dim, num_heads, head_dim, ff_dim, dropout_rate=0.2):
            super(TransformerBlockModel.CustomModel.TransformerBlock, self).__init__()
            self.attention = nn.MultiheadAttention(embed_dim=num_heads * head_dim, num_heads=num_heads, dropout=dropout_rate)
            self.norm1 = nn.LayerNorm(input_dim)  # LayerNorm based on input_dim
            self.ff = nn.Sequential(
                nn.Linear(input_dim, ff_dim),
                nn.ReLU(),
            )
            self.norm2 = nn.LayerNorm(input_dim)  # LayerNorm based on input_dim
            self.dropout = nn.Dropout(dropout_rate)
    
        def forward(self, x):
            # Attention block
            attn_output, _ = self.attention(x, x, x)  # x: (seq_len, batch_size, input_dim)
            x = self.norm1(x + self.dropout(attn_output))  # Residual + Norm
            
            # Feed-forward block
            ff_output = self.ff(x)
            x = self.norm2(x + self.dropout(ff_output))  # Residual + Norm
        
            return x

    def __init__(self, input_dim, num_heads, head_dim, dropout_rate=0.2):
        super(TransformerBlockModel.CustomModel, self).__init__()
        #self.seq_length = seq_length  # Asumiendo una longitud fija de secuencia para simplificar
        self.num_heads = num_heads
        self.head_dim = head_dim
        
        # Proyección de entrada
        self.input_projection = nn.Linear(input_dim, num_heads * head_dim)
        
        self.attention_blocks = nn.ModuleList([
          TransformerBlockModel.CustomModel.TransformerBlock(input_dim=num_heads * head_dim,
                                        num_heads=num_heads,
                                        head_dim=head_dim,
                                        ff_dim=num_heads * head_dim,
                                        dropout_rate=dropout_rate)
        ])
    
        # Capas lineales individuales para cada posición de la secuencia
        # Esto es un cambio respecto al código original para aplicar una capa lineal por posición de salida
        self.positionwise_linear = nn.Linear(num_heads * head_dim, 1)
        
        # Capa de salida final, después de un flatten, para aplicar Softmax
        # Nota: Softmax se aplica después del flatten, por lo tanto no se define aquí como una capa pero sí en el forward

    def generate_attention_mask(self, x, padding_value=0):
      # Identificar posiciones de padding en x
      mask = (x.sum(dim=-1) == padding_value)  # Asumiendo que el padding se puede identificar sumando los valores de la característica y comparando con 0
      mask = mask.to(dtype=torch.bool)  # Convierte a bool para usar como máscara
      # PyTorch espera una máscara con True y False donde True indica donde aplicar la máscara
      return mask
    
    
    def forward(self, x, seq_lengths=10, return_probabilities=False):
      # x: [batch_size, seq_length, input_dim]
      x = x.float()
    
      max_len = x.shape[1]
    
      # Generar máscara de atención basada en las longitudes de las secuencias
      attn_mask = None
    
      # Aplicar proyección de entrada
      x_proj = self.input_projection(x)
      attn_output = x_proj
      x_proj = x_proj.permute(1, 0, 2)  # Reordenar para multihead_attention: [seq_length, batch_size, num_heads*head_dim]
    
    
      # Aplicar atención multi-cabeza
      attn_output = self.attention_blocks[0](x_proj)
      attn_output = attn_output.permute(1, 0, 2)  # Reordenar de vuelta: [batch_size, seq_length, num_heads*head_dim]
    
      # Aplicar la capa lineal posición por posición
      # Usamos una capa lineal que se aplica a cada vector de salida de la atención de forma independiente
      positionwise_output = self.positionwise_linear(attn_output)
    
      # Flatten
      flat_output = positionwise_output.view(positionwise_output.size(0), -1)  # [batch_size, seq_length]
    
      # Softmax
      if return_probabilities:
        output = F.softmax(flat_output, dim=-1)
        return output
      else: #return logits
        return flat_output

  def load_model(self):
    self.model = TransformerBlockModel.CustomModel(input_dim=self.input_dim, num_heads=self.num_heads, head_dim=self.head_dim)
    self.model = self.model.to(self.device)


In [64]:
## Transformer Block Multicapa

In [65]:
class MultilayerTransformerBlockModel(TrainableModel):
  class CustomModel(TransformerBlockModel.CustomModel):
      def __init__(self, input_dim, num_heads, head_dim, dropout_rate=0.2):
          super().__init__(input_dim, num_heads, head_dim, dropout_rate)
          self.attention_blocks = nn.ModuleList([TransformerBlockModel.CustomModel.TransformerBlock(input_dim=num_heads * head_dim,
                                            num_heads=num_heads,
                                            head_dim=head_dim,
                                            ff_dim=num_heads * head_dim,
                                            dropout_rate=dropout_rate)]*6)
      def forward(self, x, seq_lengths=10, return_probabilities=False):
          # x: [batch_size, seq_length, input_dim]
          x = x.float()

          max_len = x.shape[1]

          # Generar máscara de atención basada en las longitudes de las secuencias
          attn_mask = None

          # Aplicar proyección de entrada
          x_proj = self.input_projection(x)
          attn_output = x_proj
          x_proj = x_proj.permute(1, 0, 2)  # Reordenar para multihead_attention: [seq_length, batch_size, num_heads*head_dim]


          # Aplicar atención multi-cabeza
          for i in range(len(self.attention_blocks)):
              attn_output = self.attention_blocks[i](x_proj)
          
          attn_output = attn_output.permute(1, 0, 2)  # Reordenar de vuelta: [batch_size, seq_length, num_heads*head_dim]

          # Aplicar la capa lineal posición por posición
          # Usamos una capa lineal que se aplica a cada vector de salida de la atención de forma independiente
          positionwise_output = self.positionwise_linear(attn_output)

          # Flatten
          flat_output = positionwise_output.view(positionwise_output.size(0), -1)  # [batch_size, seq_length]

          # Softmax
          if return_probabilities:
            output = F.softmax(flat_output, dim=-1)
            return output
          else: #return logits
            return flat_output
  def __init__(self):
    super().__init__()
  def load_model(self):
    self.model = self.CustomModel(input_dim=self.input_dim, num_heads=self.num_heads, head_dim=self.head_dim)
    self.model = self.model.to(self.device)

In [66]:
# import pandas as pd
# df = pd.DataFrame(columns=["Model Name", "cities", "iter", "Epoch", "Training Loss", "Training Accuracy", "Validation Loss", "Validation Accuracy"])
# df

# dfb = pd.DataFrame(columns=["Model Name", "cities", "iter", "avg path cost"])

# import gc
# models = [BaseModel(), TransformerBlockModel(), MultilayerTransformerBlockModel()]
# # for city_count in [10, 50, 100, 500]:
# num_iters=10
# eval_count = 50
# # for city_count in [50]:

# from eda.TSP import TSP_Instance, TSP_Environment, TSP_State, evalConstructiveActions, plot_tour
# from eda.agents import SingleAgentSolver, GreedyAgent
# import numpy as np


# class ModelEvalActions():
#   def __init__(self, model):
#     self.model=model

#   # permite evaluar acctiones de varios estados a la vez
#   # para optimizar los cáluclos del modelo
#   def __call__(self, states, env):
#     single_state = False
#     if not isinstance(states, list):
#       single_state=True
#       states = [states]

#     evals = [list() for _ in states]
#     vecSeqs=[]; move2idx =[]

#     for state in states:
#       vecSeq, _, mov2idx = state.state2vecSeq()
#       vecSeqs.append(vecSeq)
#       move2idx.append(mov2idx)
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     predictions = self.model(torch.tensor(vecSeqs).to(device), return_probabilities=True)

#     for k in range(len(states)):
#       state = states[k]
#       for action in env.gen_actions(state, "constructive"):
#           idx = move2idx[k][action[1]] #mapping from move to output index (model)
#           evals[k].append((action,predictions[k][idx]))

#     if single_state: return evals[0]
#     else: return evals

# for iter in range(num_iters):
#     X = None
#     Y = None
#     for model in models:
#         if X == None and Y == None:
#             print(f"Generando datos iteración i={iter}")
#             X, Y = model.generate_data(use_progress_bar=True)
#             X = X.to(model.device)
#             Y = Y.to(model.device)
        
#         name = type(model).__name__
#         print("Entrenando", name, "con n=", model.city_count)
#         trained_model_df, eval_model_df = model.train(X, Y, num_iter=iter, use_progress_bar=True)
#         pd.concat([df, trained_model_df])
#         # Evaluación post entrenamiento
#         print("Evaluando modelo: Generando 20 instancias")
#         instances = [
#             TSP_Instance(np.random.rand(model.city_count, 2)) for _ in tqdm(
#                 range(eval_count), desc="Instancias", unit="instance", position=0, leave=True)
#         ]
#         greedy = SingleAgentSolver (env,GreedyAgent(ModelEvalActions(model.model)))
#         solutions = []
#         for instance in tqdm(instances, desc="Solving Instances", unit="instance", position=0, leave=True):
#             solution, *_ = greedy.solve(TSP_State(instance, visited=[0]))
#             solutions.append(solution.cost)
#         model.unload_model()

#         solutions_prom = sum(solutions) / len(solutions)
#         dfb = pd.concat([dfb, pd.DataFrame([{
#             "Model Name": type(model).__name__,
#             "iter": iter,
#             "cities": model.city_count,
#             "avg path cost": solutions_prom
#         }])])
        
#     del X
#     del Y
#     X = None
#     Y = None


# df

In [67]:
df.to_csv("out2.csv")

NameError: name 'df' is not defined

In [68]:
dfb.groupby(["Model Name", "cities"]).mean()

NameError: name 'dfb' is not defined

In [69]:
df

NameError: name 'df' is not defined

In [70]:
train_df = pd.DataFrame(columns=["model_name", "iter", "epoch", "tr_loss", "tr_acc", "val_loss", "val_acc"])
train_df

,model_name,iter,epoch,tr_loss,tr_acc,val_loss,val_acc


In [71]:
eval_df = pd.DataFrame(columns=["model_name", "iter", "epoch", "cost"])
eval_df

,model_name,iter,epoch,cost


In [73]:
from sklearn.model_selection import KFold

from eda.TSP import TSP_Instance, TSP_Environment, TSP_State, evalConstructiveActions, plot_tour
from eda.agents import SingleAgentSolver, GreedyAgent
nb_cities = 50
nb_samples=20000
nb_eval = 50
k_folds=10
iter=0
models = [BaseModel(), TransformerBlockModel(), MultilayerTransformerBlockModel()]
class ModelEvalActions():
  def __init__(self, model):
    self.model=model

  # permite evaluar acctiones de varios estados a la vez
  # para optimizar los cáluclos del modelo
  def __call__(self, states, env):
    single_state = False
    if not isinstance(states, list):
      single_state=True
      states = [states]

    evals = [list() for _ in states]
    vecSeqs=[]; move2idx =[]

    for state in states:
      vecSeq, _, mov2idx = state.state2vecSeq()
      vecSeqs.append(vecSeq)
      move2idx.append(mov2idx)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    predictions = self.model(torch.tensor(vecSeqs).to(device), return_probabilities=True)

    for k in range(len(states)):
      state = states[k]
      for action in env.gen_actions(state, "constructive"):
          idx = move2idx[k][action[1]] #mapping from move to output index (model)
          evals[k].append((action,predictions[k][idx]))

    if single_state: return evals[0]
    else: return evals


for model in models:
        print(f"Iteration {iter}: Preparing dataset")

        # Generate data once per iteration
        print(f"Generating data for iteration {iter}")
        X, Y = model.generate_data(use_progress_bar=True)
        X = X.to(model.device)
        Y = Y.to(model.device)

        kfold = KFold(n_splits=k_folds, shuffle=True, random_state=iter)
        fold = 0

        for train_idx, val_idx in kfold.split(X):
            fold += 1

            print(f"Training {type(model).__name__} on fold {fold}")

            # Split data into train and validation sets
            X_train, X_val = X[train_idx], X[val_idx]
            Y_train, Y_val = Y[train_idx], Y[val_idx]

            model.load_model()
            # Train the model
            trained_model_df, eval_model_df = model.train(X_train, Y_train, X_val, Y_val, use_progress_bar=True, num_iter = fold +1)
                        # Log training metrics
            for _, row in trained_model_df.iterrows():
                train_df = pd.concat([train_df, pd.DataFrame([{
                    "model_name": type(model).__name__,
                    "iter": row['iter'],
                    "epoch": row["Epoch"],
                    "tr_loss": row["Training Loss"],
                    "tr_acc": row["Training Accuracy"],
                    "val_loss": row["Validation Loss"],
                    "val_acc": row["Validation Accuracy"]
                }])])
            for _, row in eval_model_df.iterrows():
                eval_df = pd.concat([eval_df, pd.DataFrame([{
                    "model_name": type(model).__name__,
                    "iter": row['iter'],
                    "epoch": row["Epoch"],
                    "cost": row['cost']
                }])])
            model.unload_model()







Iteration 0: Preparing dataset
Generating data for iteration 0


Generating data: 100%|████████████████████| 20000/20000 [00:09<00:00, 2101.98sample/s]


Training BaseModel on fold 1


Training Epochs:   0%|                                      | 0/10 [00:00<?, ?epoch/s]

Entrenando modelo...


...

C:\Users\krr\AppData\Local\Temp\ipykernel_28436\1232451449.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([{
Solving Instances:   0%|                                 | 0/50 [00:00<?, ?instance/s]C:\Users\krr\AppData\Local\Temp\ipykernel_28436\1232451449.py:105: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfb = pd.concat([dfb, pd.DataFrame([{
Training Epochs: 100%|█████████████████████████████| 10/10 [02:33<00:00, 15.35s/epoch]


Training MultilayerTransformerBlockModel on fold 10


Training Epochs:   0%|                                      | 0/10 [00:00<?, ?epoch/s]

Entrenando modelo...


C:\Users\krr\AppData\Local\Temp\ipykernel_28436\1232451449.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([{
Solving Instances:   0%|                                 | 0/50 [00:00<?, ?instance/s]C:\Users\krr\AppData\Local\Temp\ipykernel_28436\1232451449.py:105: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfb = pd.concat([dfb, pd.DataFrame([{
Training Epochs: 100%|█████████████████████████████| 10/10 [02:31<00:00, 15.19s/epoch]


In [74]:
train_df

,model_name,iter,epoch,tr_loss,tr_acc,val_loss,val_acc
0,BaseModel,2,1,1.543483,57.255556,0.634475,82.10
0,BaseModel,2,2,0.646747,79.283333,0.585470,82.80
0,BaseModel,2,3,0.601239,80.800000,0.547972,83.10
0,BaseModel,2,4,0.586964,80.944444,0.554502,82.60
0,BaseModel,2,5,0.578269,81.116667,0.547895,83.00
...,...,...,...,...,...,...,...
0,MultilayerTransformerBlockModel,11,6,0.563725,81.733333,0.552865,82.35
0,MultilayerTransformerBlockModel,11,7,0.556832,81.938889,0.565708,82.45
0,MultilayerTransformerBlockModel,11,8,0.574137,81.450000,0.550217,82.40
0,MultilayerTransformerBlockModel,11,9,0.551331,81.688889,0.545964,82.85


In [57]:
eval_df

,model_name,iter,epoch,cost
0,BaseModel,1,9,6.742856
0,BaseModel,2,9,6.513208
0,BaseModel,3,9,6.817352
0,BaseModel,4,9,6.544759
0,BaseModel,5,9,6.461715
0,BaseModel,6,9,5.633680
0,BaseModel,7,9,6.411649
0,BaseModel,8,9,6.015264
0,BaseModel,9,9,6.622939
0,BaseModel,10,9,6.230760


In [76]:
ex_traindf = train_df.reset_index().drop('index', axis=1)
ex_evaldf = eval_df.reset_index().drop('index', axis=1)
ex_evaldf


,model_name,iter,epoch,cost
0,BaseModel,1,0,6.516486
1,BaseModel,1,0,6.876385
2,BaseModel,1,0,5.985434
3,BaseModel,1,0,6.970155
4,BaseModel,1,0,6.909487
...,...,...,...,...
14995,MultilayerTransformerBlockModel,10,9,6.369833
14996,MultilayerTransformerBlockModel,10,9,5.440275
14997,MultilayerTransformerBlockModel,10,9,5.995565
14998,MultilayerTransformerBlockModel,10,9,6.704208


In [77]:
ex_traindf.to_csv("train_df.csv")
ex_evaldf.to_csv("eval_df.csv")